In [ ]:
import os, sys
sys.path.append(os.path.abspath(".."))

import requests
import scrapy
import time
import pandas as pd
from pprint import pprint
import re
import json
import datetime

from scrapers import staff

In [ ]:
data = staff.main()

In [ ]:
with open("../data/postings/staff_08.10.21.json", "r") as f:
    dat = json.load(f)

len(dat["URL"])

In [ ]:
len(data.keys())

In [ ]:
all_default_keys = (
    "Company_Title", "Company_URL", "URL", "Total_views", "Followers",
    "Active_Jobs", "Jobs_History", "Job_Views", "Job_Title",
    "Application_Deadline", "Industry", "Employment_term",
    "Job_Category", "Job_type", "Job_Location",
    "Job_description", "Job_responsibilities",
    "Required_qualifications", "Required_candidate_level",
    "Salary", "Additional_information",
    "Professional_skills", "Soft_skills"
)

In [ ]:
new_keys = [k for k in data.keys() if k not in all_default_keys]

In [ ]:
len(new_keys)

In [ ]:
sorted([(k, len(data[k])) for k in new_keys], key=lambda x: x[1])[::-1]

In [ ]:
data["Additional_information"]

In [ ]:
rs = requests.get("https://staff.am/en/senior-uiux-designer-112")
response = scrapy.http.HtmlResponse(url=rs.url, body=rs.text, encoding="utf-8")

In [ ]:
staff.get_info_from_one_posting("https://staff.am/en/senior-uiux-designer-112")

In [ ]:
scrape = staff.crawl_all_postings(["https://staff.am/en/senior-engineer-ii-c", "https://staff.am/en/senior-uiux-designer-112"])

In [ ]:
arr = [f"{i}th val" for i in range(100)]

In [ ]:
import time
from rich.progress import track
from rich import progress_bar

In [ ]:
for i in track(arr):
    time.sleep(0.1)